In [ ]:
# Widgets
from IPython.display import display, clear_output
import ipywidgets as widgets
from ipywidgets import Layout, interact

# Imports
import pandas as pd
import numpy as np
import math

import io
import requests

import time

import gspread
from oauth2client.service_account import ServiceAccountCredentials

scope = "https://spreadsheets.google.com/feeds"
credentials = ServiceAccountCredentials.from_json_keyfile_name("matcher-272116-8801bce55bcb.json", scope)
gs = gspread.authorize(credentials)

data = None

# Use journey data from express guaranteed Google Sheet?

In [ ]:
express_guaranteed = widgets.Dropdown(
    options = ["Select option", "Cyfe Board (May be empty now)", "Express Guaranteed Sheet"],
    value = "Select option",
    description = "Where to pull journey data from?",
    layout = Layout(width = "25%"),
    style = {"description_width":"initial"})

express_guaranteed_button = widgets.Button(description = "Load", layout = Layout(width = "80px"))
out_journey = widgets.Output()

def load_journey_data(_) :
    global data, continue_load
    with out_journey :
        clear_output()
        print("Loading journeys", end = "\r")
        
        if express_guaranteed.value == "Express Guaranteed Sheet" :
            url = "https://docs.google.com/spreadsheets/d/1o7jEklbPYk-7So0UALnt8dZCYlIbdyXhzsgm9MHf82I/edit#gid=0"

            express_guaranteed_sheet = gs.open_by_url(url).worksheet("journeys").get_all_values()
            data = pd.DataFrame(express_guaranteed_sheet[1:], columns = express_guaranteed_sheet[0])
            data = data.replace("", np.nan)
            print("{0} journeys loaded, {1} expressions    ".format(data["journey_id"].nunique(), len(data["name#reg"].dropna())))
        elif express_guaranteed.value == "Cyfe Board (May be empty now)" :
            url = "https://www.anyvan.com/board/cyfe/express_interest_expresses"
            s = requests.get(url).content

            try :
                data = pd.read_csv(io.StringIO(s.decode("utf-8"))).sort_values(by = ["start_end"], ascending = False)
                print("{0} journeys loaded, {1} expressions    ".format(data["journey_id"].nunique(), len(data["name#reg"].dropna())))
            except :
                data = None
                print("No journeys available")
        else :
            data = None
            print("Select one of the options")
                
express_guaranteed_button.on_click(load_journey_data)

display(widgets.VBox([widgets.HBox([express_guaranteed, express_guaranteed_button]), out_journey]))

# TPs

In [ ]:
TPs = None

TPs_button = widgets.Button(description = "Load TPs", layout = Layout(width = "80 px"))
out_TPs = widgets.Output()

def load_TPs(_) :
    global TPs
    with out_TPs :
        clear_output()
        print("Loading TPs", end = "\r")
        
        # Returns
        url = "https://docs.google.com/spreadsheets/d/1_syJVv1Uni8oSa9BhtAjVcl2EZYZmJn03VKg9vofcD4/edit#gid=0"

        return_journey_converter_sheet = gs.open_by_url(url).worksheet("Input").get_all_values()[15:]
        returns = pd.DataFrame(return_journey_converter_sheet[1:], columns = return_journey_converter_sheet[0])
        returns = returns.drop(columns = returns.columns[16:])
        returns["nickname"].replace("", float("NaN"), inplace = True)
        returns = returns.dropna()

        # Providers
        sizes = {"Luton Van" : 20, 
                 "LWB up to 4m" : 10.5,
                 "MWB up to 3m" : 7.5,
                 "Not applicable" : 12,
                 "XLWB up 4m+" : 14.5,
                 "SWB up to 2.4m" : 5.5,
                 "Small Van" : 8}

        url = "https://docs.google.com/spreadsheets/d/1t0L9sgsUt-BT7wJWBNZ4fNY-Bo5GNytXP1wmkejFVw0/edit#gid=1205069010"

        TP_log_providers_sheet = gs.open_by_url(url).worksheet("format2").get_all_values()[7:]
        TP_log_providers = pd.DataFrame(TP_log_providers_sheet[1:], columns = TP_log_providers_sheet[0])
        providers = TP_log_providers.where(np.logical_and(TP_log_providers["vehicle"] != "N/A", np.logical_and(TP_log_providers["TP & REG"] != "AVStore #YN19FHD", np.logical_and(TP_log_providers["TP & REG"] != "1London #YN19FHF", TP_log_providers["TP & REG"] != "DWB91 #YN19FHD")))).dropna(how = "all").reset_index(drop = True).drop(columns = TP_log_providers.columns[-1])
        date = providers.columns[-1]

        providers["Size"] = [sizes[vehicle] for vehicle in providers["vehicle"]]
        providers["National?"] = [int(math.ceil(55 + (100 - float(nuts2c)) / 2 + (2000 if code.endswith("N") else 0))) for nuts2c,code in zip(providers["nuts2c"], providers[date])]
        providers["1Man"] = [1 if (code.startswith("1") or code.startswith("O")) else 0 for code in providers[date]]
        providers["2Man"] = [2 if (code.startswith("2") or code.startswith("O")) else 0 for code in providers[date]]
        providers["returns"] = [1 if TP in returns["name#reg"] else 0 for TP in providers["TP & REG"]]
        providers["guaranteed"] = [1 if isreturn == 1 else -1 for isreturn in providers["returns"]]
        providers["special code"] = ["2man-TL-LDN-cc" if (TwoMan == 2 and TL == "1") else "TL-LDN-cc" if (TL == "1") else "2man-LDN-cc" if (TwoMan == 2) else "LDN-cc" for TwoMan,TL in zip(providers["2Man"], providers["TL"])]
        providers["capacity"] = [16 if size >= 16 else size for size in providers["Size"]]

        providers = providers.sort_values(by = ["TP & REG"]).reset_index(drop = True)
        providers["index"] = providers.index

        # Expressing TPs
        columns = ["TP", "lat", "lng", "expressions", "guaranteed", "boooked"]
        expressing_TPs = pd.DataFrame(columns = columns)

        if data is not None :
            for TP in data["name#reg"].dropna().unique() :
                data_TP = data.where(data["name#reg"] == TP).dropna(how = "all")
                lat = round(float(data_TP.iloc[0]["veh_lat"]), 2)
                lng = round(float(data_TP.iloc[0]["veh_lng"]), 2)
                expressions = len(data_TP.index)
                guaranteed = 1 if (TP in providers["TP & REG"] or TP in returns["name#reg"]) else 0
                booked = 1 if TP in returns["name#reg"] else 0
                expressing_TPs = expressing_TPs.append(pd.DataFrame([[TP, lat, lng, expressions, guaranteed, booked]], columns = columns))
        else :
            print("No journeys available")

        expressing_TPs = expressing_TPs.sort_values(by = ["TP"]).reset_index(drop = True)
        expressing_TPs["index"] = expressing_TPs.index

        # Combine TPs
        TP_namereg = np.append(providers["TP & REG"], expressing_TPs["TP"])
        TP_nickname = [name.split(" ")[0] for name in TP_namereg]
        TP_lat = np.append(providers["Lat"], expressing_TPs["lat"])
        TP_lng = np.append(providers["Lng"], expressing_TPs["lng"])
        TP_TL = np.append(providers["TL"], expressing_TPs["expressions"])
        TP_capacity = np.append(providers["Size"], np.full(len(expressing_TPs.index), np.nan))
        TP_maxkm = np.append(providers["National?"], np.full(len(expressing_TPs.index), np.nan))
        TP_1man = np.append(providers["1Man"], np.full(len(expressing_TPs.index), np.nan))
        TP_2man = np.append(providers["2Man"], np.full(len(expressing_TPs.index), np.nan))
        TP_guaranteed = np.append(providers["guaranteed"], expressing_TPs["guaranteed"])
        TP_provider = np.append(np.ones(len(providers.index), dtype = bool), np.zeros(len(expressing_TPs.index), dtype = bool))

        columns = ["TP", "nickname", "lat", "lng", "TL", "capacity", "maxkm", "1man", "2man", "guaranteed", "provider"]
        TPs = pd.DataFrame(zip(TP_namereg, TP_nickname, TP_lat, TP_lng, TP_TL, TP_capacity, TP_maxkm, TP_1man, TP_2man, TP_guaranteed, TP_provider), columns = columns)
        TPs = TPs.where(np.logical_and(TPs["guaranteed"] <= 0, ~TPs["TP"].duplicated())).dropna(how = "all").sort_values(by = ["provider", "TP"], ascending = [False, True])
        
        print("{0} TPs loaded      ".format(len(TPs["TP"].dropna())))
    
TPs_button.on_click(load_TPs)

widgets.VBox([TPs_button, out_TPs])

# Calculate iarray

In [ ]:
iarray_button = widgets.Button(description = "Calculate IArray", layout = Layout(width = "150px"))
out_iarray = widgets.Output()

def distance_funct(start_lat, start_lng, lat, lng) :
    return 2 * 6371 * np.arcsin(np.sqrt(np.sin((start_lat - lat) * np.pi / 360) ** 2 
                                           + np.cos(start_lat * np.pi / 180) 
                                           * np.cos(lat * np.pi / 180) 
                                           * np.sin((start_lng - lng) * np.pi / 360) ** 2))

iarray = None
journeys_data = None

def calc_iarray(_) :
    global iarray, journeys_data
    with out_iarray :
        clear_output()
        if data is not None :
            start_time = time.time()

            journeys = data["journey_id"].unique()

            columns = ["journey_id", "tp_amount", "vat", "vat_pef_multiplier"]
            journeys_data = pd.DataFrame(columns = columns)
            iarray = np.zeros((len(TPs.index), len(journeys)), dtype = float)

            vat_impact = 1000

            iarray_progress = widgets.IntProgress(
                value = 0,
                min = 0,
                max = len(journeys) - 1,
                step = 1,
                description = "Calculating",
                bar_style = "success",
                orientation = "horizontal",
                layout = Layout(width = "40%"),
                style = {"description_width":"initial"}
            )

            display(widgets.HBox([iarray_progress]))

            for i in range(len(journeys)) :
                journey = data.where(data["journey_id"] == journeys[i]).dropna(how = "all")
                journeys_data = journeys_data.append(pd.DataFrame([[str(int(journey["journey_id"].iloc[0])), journey["tp_amount"].iloc[0], float(journey["tp_amount"].iloc[0]) / 6 / 1.2, 1 - np.tanh(float(journey["tp_amount"].iloc[0]) / 6 / 1.2 / vat_impact)]], columns = columns))

                for j in range(len(TPs.index)) :
                    if int(TPs["guaranteed"].iloc[j]) == -1 :
                        if TPs["TP"].iloc[j] in journey["name#reg"].values :
                            journey_subset = journey.where(journey["name#reg"] == TPs["TP"].iloc[j]).dropna(how = "all").iloc[0]
                            if float(journey_subset["min_dist"]) >= 5 :
                                iarray[j, i] = 5
                            else :
                                iarray[j, i] = float(journey_subset["min_dist"])
                        else :
                            if float(journey["max_m3"].iloc[0]) <= float(TPs["capacity"].iloc[j]) :
                                if float(journey["start_end"].iloc[0]) <= float(TPs["maxkm"].iloc[j]) :
                                    if int(journey["number_of_men"].iloc[0]) == int(TPs["1man"].iloc[j]) or int(journey["number_of_men"].iloc[0]) == int(TPs["2man"].iloc[j]) :
                                        if int(journey["tail_lift"].iloc[0]) <= int(TPs["TL"].iloc[j]) :
                                            distance = distance_funct(float(journey["start_lat"].iloc[0]), float(journey["start_lng"].iloc[0]), float(TPs["lat"].iloc[j]), float(TPs["lng"].iloc[j]))
                                            if distance <= (float(TPs["maxkm"].iloc[j]) if float(TPs["maxkm"].iloc[j]) < 2000 else float(TPs["maxkm"].iloc[j]) - 2000) :
                                                iarray[j, i] = distance
                    elif TPs["TP"].iloc[j] in journey["name#reg"].values :
                        iarray[j, i] = float(journey.where(journey["name#reg"] == TPs["TP"].iloc[j]).dropna(how = "all")["min_dist"].iloc[0])
                iarray_progress.value = i

            print("Took {0:.3f} secs to run".format(time.time() - start_time))

        else :
            print("No journeys available")

        journeys_data = journeys_data.reset_index(drop = True)
    
iarray_button.on_click(calc_iarray)

widgets.VBox([iarray_button, out_iarray])

In [ ]:
if data is not None :
    journey_menu = widgets.Dropdown(
        options = np.insert(np.array(journeys_data.sort_values(by = "journey_id")["journey_id"]), 0, "Select journey"),
        value = "Select journey",
        description = "Journey:")
else :
    journey_menu = widgets.Dropdown(
        options = ["Select journey"],
        value = "Select journey",
        description = "Journey:")
    
TP_menu = widgets.Dropdown(
    options = np.insert(np.array(TPs["TP"]), 0, "Select TP"),
    value = "Select TP",
    description = "TP:")

journey_button = widgets.Button(description = "Find", layout = Layout(width = "80px"))
TP_button = widgets.Button(description = "Find", layout = Layout(width = "80px"))
out_display = widgets.Output()

def journey_display(_) :
    TP_menu.value = "Select TP"
    
    with out_display :
        clear_output()
        if journey_menu.value != "Select journey" and journey_menu.value != None:
            print("Journey: {0}\n".format(journey_menu.value))
            for TP, iarray_val in list(zip(TPs["TP"], iarray.transpose()[np.argwhere(np.array(journeys_data["journey_id"]) == journey_menu.value)][0][0])) :
                print("{0:20} -> {1}".format(TP, iarray_val))

def TP_display(_) :
    journey_menu.value = "Select journey"
    
    with out_display :
        clear_output()
        if TP_menu.value != "Select TP" and TP_menu.value != None:
            print("TP: {0}\n".format(TP_menu.value))
            for journey, iarray_val in list(zip(journeys_data["journey_id"], iarray[np.argwhere(np.array(TPs["TP"]) == TP_menu.value)][0][0])) :
                print("{0:20} -> {1}".format(journey, iarray_val))

journey_button.on_click(journey_display)
TP_button.on_click(TP_display)

widgets.VBox([widgets.HBox([journey_menu, journey_button, TP_menu, TP_button]), out_display])

In [ ]:
if data is not None :
    journey_menu_edit = widgets.Dropdown(
        options = np.insert(np.array(journeys_data.sort_values(by = "journey_id")["journey_id"]), 0, "Select journey"),
        value = "Select journey",
        description = "Journey:",
        style = {"description_width":"initial"})
else :
    journey_menu_edit = widgets.Dropdown(
        options = ["Select journey"],
        value = "Select journey",
        description = "Journey:",
        style = {"description_width":"initial"})
    
TP_menu_edit = widgets.Dropdown(
    options = np.insert(np.array(TPs["TP"]), 0, "Select TP"),
    value = "Select TP",
    description = "TP:",
    style = {"description_width":"initial"})

iarray_edit_box = widgets.FloatText(
    value = 0,
    description = "New IArray value:",
    style = {"description_width":"initial"})

iarray_edit_button = widgets.Button(description = "Update IArray", style = {"description_width":"initial"})
out_edit = widgets.Output()

def iarray_edit(_) :
    with out_edit :
        clear_output()
        
        if journey_menu_edit.value == "Select journey" or TP_menu_edit.value == "Select TP" :
            print("Select a journey and a TP")
        elif iarray_edit_box.value is None or iarray_edit_box.value < 0 :
            print("Select an appropriate value for the iarray edit")
        else :
            old_iarray_value = iarray[np.argwhere(np.array(TPs["TP"]) == TP_menu_edit.value), np.argwhere(np.array(journeys_data["journey_id"]) == journey_menu_edit.value)][0][0]
            iarray[np.argwhere(np.array(TPs["TP"]) == TP_menu_edit.value), np.argwhere(np.array(journeys_data["journey_id"]) == journey_menu_edit.value)] = float(iarray_edit_box.value)
            print("TP: {0}\nJourney: {1}\nOld IArray Value: {2}\nNew Iarray Value: {3}".format(TP_menu_edit.value, journey_menu_edit.value, old_iarray_value, iarray_edit_box.value))
            
iarray_edit_button.on_click(iarray_edit)

widgets.VBox([widgets.HBox([journey_menu_edit, TP_menu_edit, iarray_edit_box, iarray_edit_button]), out_edit])

# Provider Stats and TP Score

In [ ]:
url = "https://www.anyvan.com/board/cyfe/express_interest_pstats"
s = requests.get(url).content

try :
    print("Loading provider stats", end = "\r")
    pstats = pd.read_csv(io.StringIO(s.decode("utf-8")))
    print("Provider stats loaded                ")
except :
    data = None
    print("No provider stats available")
    
score_weight = {"rank_vat" : 10,
                "rank_g" : 25,
                "rank_wps" : 1,
                "rank_rating" : 2,
                "rank_dsw" : 1,
                "rank_ssw" : 2,
                "rank_exp" : 1}

TP_pstats = TPs.merge(pstats, how = "left", left_on = "nickname", right_on = "nickname")

TP_pstats["express"] = [10 if guaranteed == -1 else TL for guaranteed, TL in zip(TP_pstats["guaranteed"], TP_pstats["TL"])]
TP_pstats["rank_vat"] = TP_pstats["vat?"].rank(method = "min", ascending = False) / len(TP_pstats.index)
TP_pstats["rank_g"] = TP_pstats["guaranteed"].rank(method = "min", ascending = False) / len(TP_pstats.index)
TP_pstats["rank_wps"] = TP_pstats["wins per session"].rank(method = "min", ascending = False) / len(TP_pstats.index)
TP_pstats["rank_rating"] = TP_pstats["avg rating"].rank(method = "min", ascending = True) / len(TP_pstats.index)
TP_pstats["rank_dsw"] = TP_pstats["days_since_win"].rank(method = "min", ascending = True) / len(TP_pstats.index)
TP_pstats["rank_ssw"] = TP_pstats["sessions_since_win"].rank(method = "min", ascending = True) / len(TP_pstats.index)
TP_pstats["rank_exp"] = TP_pstats["express"].rank(method = "min", ascending = True) / len(TP_pstats.index)

TP_pstats["weighted_score"] = TP_pstats["rank_vat"] * score_weight["rank_vat"] + TP_pstats["rank_g"] * score_weight["rank_g"] + TP_pstats["rank_wps"] * score_weight["rank_wps"] + TP_pstats["rank_rating"] * score_weight["rank_rating"] + TP_pstats["rank_dsw"] * score_weight["rank_dsw"] + TP_pstats["rank_ssw"] * score_weight["rank_ssw"] + TP_pstats["rank_exp"] * score_weight["rank_exp"]
weighted_score_max = TP_pstats["weighted_score"].max()

TP_pstats["weighted_rank"] = [0.75 if guaranteed == -1 else (1.5 - weighted_score / weighted_score_max) for guaranteed, weighted_score in zip(TP_pstats["guaranteed"], TP_pstats["weighted_score"])]

print("TP Score calculated")

# Marray

In [ ]:
marray_button = widgets.Button(description = "Calculate MArray", layout = Layout(width = "150px"))
out_marray = widgets.Output()

marray = None

def calc_marray(_) :
    global marray
    with out_marray :
        marray = iarray.copy()

        for i in range(len(journeys_data.index)) :
            for j in range(len(TP_pstats.index)) :
                if iarray[j, i] > 0 :
                    marray[j, i] = (iarray[j,i] + (100 if int(TP_pstats["guaranteed"].iloc[j]) >= 0 else 0)) * float(TP_pstats["weighted_rank"].iloc[j]) * (float(journeys_data["vat_pef_multiplier"].iloc[i]) if int(TP_pstats["vat?"].iloc[j]) == 0 else 1)
                    
        print("MArray calculated")
                    
marray_button.on_click(calc_marray)

widgets.VBox([marray_button, out_marray])

In [ ]:
if data is not None :
    journey_menu_marray = widgets.Dropdown(
        options = np.insert(np.array(journeys_data.sort_values(by = "journey_id")["journey_id"]), 0, "Select journey"),
        value = "Select journey",
        description = "Journey:")
else :
    journey_menu_marray = widgets.Dropdown(
        options = ["Select journey"],
        value = "Select journey",
        description = "Journey:")
    
TP_menu_marray = widgets.Dropdown(
    options = np.insert(np.array(TP_pstats["TP"]), 0, "Select TP"),
    value = "Select TP",
    description = "TP:")

journey_button_marray = widgets.Button(description = "Find", layout = Layout(width = "80px"))
TP_button_marray = widgets.Button(description = "Find", layout = Layout(width = "80px"))
out_display_marray = widgets.Output()

def journey_display_marray(_) :
    TP_menu.value = "Select TP"
    
    with out_display_marray :
        clear_output()
        if journey_menu_marray.value != "Select journey" and journey_menu_marray.value != None:
            print("Journey: {0}\n".format(journey_menu_marray.value))
            for TP, marray_val in list(zip(TP_pstats["TP"], marray.transpose()[np.argwhere(np.array(journeys_data["journey_id"]) == journey_menu_marray.value)][0][0])) :
                print("{0:20} -> {1}".format(TP, marray_val))

def TP_display_marray(_) :
    journey_menu.value = "Select journey"
    
    with out_display_marray :
        clear_output()
        if TP_menu_marray.value != "Select TP" and TP_menu_marray.value != None:
            print("TP: {0}\n".format(TP_menu_marray.value))
            for journey, marray_val in list(zip(journeys_data["journey_id"], marray[np.argwhere(np.array(TPs["TP"]) == TP_menu_marray.value)][0][0])) :
                print("{0:20} -> {1}".format(journey, marray_val))

journey_button_marray.on_click(journey_display_marray)
TP_button_marray.on_click(TP_display_marray)

widgets.VBox([widgets.HBox([journey_menu_marray, journey_button_marray, TP_menu_marray, TP_button_marray]), out_display_marray])

# Done (Need to add matcher capability now)